In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
from sklearn.feature_extraction.text import CountVectorizer
from wilds.common.data_loaders import get_train_loader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

In [ ]:
def CleanText(text):
    text = re.sub(r'''[\[|\]]''', "", text).split()
    text = np.array(text, dtype="float64")
    return text

In [ ]:
df = pd.read_csv("../CSVFiles/small50000DomainDataBertweetEmbedded.csv")

In [ ]:
df = df.loc[:, ["comment_text", "split", "toxicity", "male", "female", "LGBTQ", "christian", "muslim", "other_religion", "black", "white"]]
df['comment_text'] = df['comment_text'].apply(lambda text: CleanText(text))

df['toxicity'] = df['toxicity'].apply(lambda text: np.round(text >= 0.5))
df['male'] = df['male'].apply(lambda x: np.round(x>=0.5))
df['female'] = df['female'].apply(lambda x: np.round(x>=0.5))
df['LGBTQ'] = df['LGBTQ'].apply(lambda x: np.round(x>=0.5))
df['christian'] = df['christian'].apply(lambda x: np.round(x>=0.5))
df['muslim'] = df['muslim'].apply(lambda x: np.round(x>=0.5))
df['other_religion'] = df['other_religion'].apply(lambda x: np.round(x>=0.5))
df['black'] = df['black'].apply(lambda x: np.round(x>=0.5))
df['white'] = df['white'].apply(lambda x: np.round(x>=0.5))

In [ ]:
# Updating values for training_data
training_data = df[df['split'] == 'train']

# Getting test_data
test_data = df[df['split'] == 'test']

# Getting validation_data
validation_data = df[df['split'] == 'val']

In [ ]:
X_train = training_data['comment_text'].values.tolist()
Y_train = training_data['toxicity'].values.tolist()

X_test = test_data['comment_text'].values.tolist()
Y_test = test_data['toxicity'].values.tolist()

X_val = validation_data['comment_text'].values.tolist()
Y_val = validation_data['toxicity'].values.tolist()

In [ ]:
class AdalineGD():
    def __init__(self, learning_rate = 0.01, n_iter = 20, w = None):
        self.w = w
        self.learning_rate = learning_rate
        self.n_iter = n_iter
        
    def CheckAccuracy(self, predictions, labels):
        acc = 0.0
        for i in range(len(predictions)):
            #print(predictions[i],labels[i])
            if (predictions[i] == labels[i]):
                acc += 1
        return acc/len(predictions)
    
    def predict(self, X_test):
        # Making sure that array is numpy array
        X_test = np.array(X_test).reshape((len(X_test), -1))
        output = np.dot(X_test, self.w)
        # Sigmoid giving 0-1 results
        
        preds = np.round(1/(1+np.exp(-output)))
        return preds
    
    def PredictTestSetDemographic(self, dfTest):
        dic = {}
        for col in dfTest.columns[3:]:
            tempSet = dfTest[dfTest[col] == 1]
            tempLabels = tempSet['toxicity'].values.tolist()
            tempVecs = tempSet['comment_text'].values.tolist()
            dic[col] = [tempVecs, tempLabels]
        
        f1_scores = []
        for col in dic:
            tempPredictions = self.predict(dic[col][0])
            currentF1 = f1_score(dic[col][1], tempPredictions, zero_division=1)
            f1_scores.append([currentF1, col])
        
        f1_scores = np.array(f1_scores)
        return f1_scores
        
    def fit(self, X_train, Y_train, X_val, Y_val):
        # Making sure that arrays are numpy arrays
        
        X_train = np.array(X_train).reshape((len(X_train), -1))
        Y_train = np.array(Y_train).reshape((len(Y_train), 1))
        
        X_val = np.array(X_val).reshape((len(X_val), -1))
        Y_val = np.array(Y_val).reshape((len(Y_val), 1))
        
        # Initializing w vector using random normal distribution
        if self.w == None:
            self.w = np.random.normal(0, 0.1, len(X_train[0])).reshape(len(X_train[0]), 1)
        
        badEpoch = 0
        self.bestW = self.w
        bestAcc = 0
        early_stopping = 150
        
        self.TrainAccArr_ = []
        self.ValAccArr_ = []
        
        # Using n epochs
        for i in range(self.n_iter):
            output = 1/(1+np.exp(-np.dot(X_train, self.w)))
            error = (Y_train - output)
            self.w += (1/len(Y_train)) * self.learning_rate * np.dot(X_train.T, error)
        
            tmpPredsVal = self.predict(X_val)
            valAcc = self.CheckAccuracy(tmpPredsVal, Y_val)
            self.ValAccArr_.append(valAcc)
            
            tmpPredsTrain = self.predict(X_train)
            TrainAcc = self.CheckAccuracy(tmpPredsTrain, Y_train)
            self.TrainAccArr_.append(TrainAcc)
            
            # Saving the best model and also checks for Early_Stopping
            if valAcc > bestAcc:
                bestAcc = valAcc
                badEpoch = 0
                self.bestW = self.w
            else:
                badEpoch += 1
            
            if badEpoch >= early_stopping:
                self.w = self.bestW                
                print("Stopped cause of bad Epoch in iteration: ", i)
                break
            #Print status
            if (i % 100) == 0:
                print('Iteration ', i, ' Done')
                
        self.TrainAccArr_ = np.array(self.TrainAccArr_)
        self.ValAccArr_ = np.array(self.ValAccArr_)
        self.w = self.bestW 
        return self

In [ ]:
model = AdalineGD(learning_rate = 0.1, n_iter = 5000)

model.fit(X_train, Y_train, X_val, Y_val)

predictions = model.predict(X_test)
Y_test = np.array(Y_test).reshape(len(Y_test), 1)

model.CheckAccuracy(predictions, Y_test)

In [ ]:
plt.figure()
plt.scatter(np.arange(0,len(model.TrainAccArr_)), model.TrainAccArr_, color='r', label='Training accuracy')
plt.scatter(np.arange(0,len(model.ValAccArr_)), model.ValAccArr_, color='g', label='Validation accuracy')
#plt.title("Training loss vs Validation loss")
plt.xlabel("Iteration")
plt.ylabel("Accuracy")
plt.legend(loc='lower right')
plt.savefig('BaselineClassifier accuracy')
plt.show()

In [ ]:
forBarPlot = model.PredictTestSetDemographic(test_data)
print(forBarPlot)
plt.figure()

x = forBarPlot[:,1]
for i in range(len(x)):
    if i%2 == 0:
        x[i] = "\n"+x[i]

y = np.array(forBarPlot[:,0].astype(float))

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, y, color=['b', 'g', 'r', 'c', 'm', 'y', 'brown', 'k'])
plt.xlabel("\nDemographic source")
plt.ylabel("F1-scores")
plt.xticks(x_pos, x)
plt.savefig('FinalF1BaselineClassifier')
plt.show()

In [ ]:
forBarPlot

In [ ]:
print(np.var(forBarPlot[:,0].astype(float)))
print(np.max(forBarPlot[:,0].astype(float)) - np.min(forBarPlot[:,0].astype(float)))

In [ ]:
print("Antal toxic i predictions: ", sum(predictions))
print("Antal toxic i test set: ", sum(Y_test))
print("Samples i test set: ", len(Y_test))